# Lab : Batch Updates to a S3 Datalake using Apache Hudi

# Merge On Read


## Table of Contents:

- 1. [Overview](#Overview)
- 2. [Merge on Read](#Merge-On-Read) 
   - 2.1 [Bulk Insert the Initial Dataset](#Bulk-Insert-the-Initial-Dataset)
   - 2.2 [Batch Upsert some records](#Batch-Upsert-some-records)
   - 2.3 [Deleting Records](#Deleting-Records.)
-3. [Hudi Compaction](#Hudi-Compaction)  



## Overview

This notebook demonstrates using PySpark on [Apache Hudi](https://aws.amazon.com/emr/features/hudi/) on Amazon EMR to insert/upsert/delete records to an S3 data lake.

Here is a good reference link to read later:

* [How Hudi Works](https://docs.aws.amazon.com/emr/latest/ReleaseGuide/emr-hudi-how-it-works.html)

This notebook covers the following concepts when writing  Merge-On-Read tables to an S3 Datalake:

- Write Hudi Spark jobs in PySpark.
- Bulk Insert the Initial Dataset.
- Sync the Hudi tables to the Hive/Glue Catalog.
- Upsert some records to a Hudi MOR table.
- Delete records from a Hudi MOR table.
- Understand Hudi Compaction



### This demo is based on Hudi version 0.8.0 and runs fine on Jupyter Notebooks connected to a 1 node (r5.4xlarge) EMR cluster with configuration listed below 

 - EMR versions 6.5.0 
 
 - Software configuration

       - Hadoop 3.2.1
       - Hive 3.1.2
       - Livy 0.7.1
       - JupyterHub 1.4.1
       - Spark 3.1.2
       
       
 - AWS Glue Data Catalog settings - Select the below listed check boxes
       - Use for Hive table metadata  
       - Use for Spark table metadata



### Connect to the Master Node of EMR cluster Using SSH :
    - ssh -i ~/xxxx.pem hadoop@<ec2-xx-xxx-xx-xx.us-west-2.compute.amazonaws.com>

    - Ensure  the below listed files are copied into HDFS.

    - hdfs dfs -copyFromLocal /usr/lib/hudi/hudi-spark-bundle.jar hdfs:///user/hadoop/

    - hdfs dfs -copyFromLocal /usr/lib/spark/external/lib/spark-avro.jar hdfs:///user/hadoop/

    - hdfs dfs -copyFromLocal /usr/share/aws/aws-java-sdk/aws-java-sdk-bundle-1.12.31.jar hdfs:///user/hadoop/
      (https://github.com/apache/hudi/issues/5053)

Let's start by initializing the Spark Session to connect this notebook to our Spark EMR cluster:

In [1]:
%%configure -f
{
    "conf":  { 
             "spark.jars":"hdfs:///user/hadoop/aws-java-sdk-bundle-1.12.31.jar, hdfs:///user/hadoop/hudi-spark-bundle.jar,hdfs:///user/hadoop/spark-avro.jar",
             "spark.sql.hive.convertMetastoreParquet":"false",     
             "spark.serializer":"org.apache.spark.serializer.KryoSerializer",
             "spark.dynamicAllocation.executorIdleTimeout": 3600,
             "spark.executor.memory": "5G",
             "spark.executor.cores": 4,
             "spark.dynamicAllocation.initialExecutors":5
           } 
}

The constants for Python to use:

In [2]:
# General Constants
HUDI_FORMAT = "org.apache.hudi"
TABLE_NAME = "hoodie.table.name"
RECORDKEY_FIELD_OPT_KEY = "hoodie.datasource.write.recordkey.field"
PRECOMBINE_FIELD_OPT_KEY = "hoodie.datasource.write.precombine.field"
OPERATION_OPT_KEY = "hoodie.datasource.write.operation"
BULK_INSERT_OPERATION_OPT_VAL = "bulk_insert"
UPSERT_OPERATION_OPT_VAL = "upsert"
DELETE_OPERATION_OPT_VAL = "delete"
BULK_INSERT_PARALLELISM = "hoodie.bulkinsert.shuffle.parallelism"
UPSERT_PARALLELISM = "hoodie.upsert.shuffle.parallelism"
S3_CONSISTENCY_CHECK = "hoodie.consistency.check.enabled"
HUDI_CLEANER_POLICY = "hoodie.cleaner.policy"
KEEP_LATEST_COMMITS = "KEEP_LATEST_COMMITS"
HUDI_COMMITS_RETAINED = "hoodie.cleaner.commits.retained"
PAYLOAD_CLASS_OPT_KEY = "hoodie.datasource.write.payload.class.key()"
EMPTY_PAYLOAD_CLASS_OPT_VAL = "org.apache.hudi.EmptyHoodieRecordPayload"

# Hive Constants
HIVE_SYNC_ENABLED_OPT_KEY="hoodie.datasource.hive_sync.enable"
HIVE_PARTITION_FIELDS_OPT_KEY="hoodie.datasource.hive_sync.partition_fields"
HIVE_ASSUME_DATE_PARTITION_OPT_KEY="hoodie.datasource.hive_sync.assume_date_partitioning"
HIVE_PARTITION_EXTRACTOR_CLASS_OPT_KEY="hoodie.datasource.hive_sync.partition_extractor_class"
HIVE_TABLE_OPT_KEY="hoodie.datasource.hive_sync.table"

# Partition Constants
NONPARTITION_EXTRACTOR_CLASS_OPT_VAL="org.apache.hudi.hive.NonPartitionedExtractor"
MULTIPART_KEYS_EXTRACTOR_CLASS_OPT_VAL="org.apache.hudi.hive.MultiPartKeysValueExtractor"
KEYGENERATOR_CLASS_OPT_KEY="hoodie.datasource.write.keygenerator.class"
NONPARTITIONED_KEYGENERATOR_CLASS_OPT_VAL="org.apache.hudi.keygen.NonpartitionedKeyGenerator"
COMPLEX_KEYGENERATOR_CLASS_OPT_VAL="org.apache.hudi.ComplexKeyGenerator"
PARTITIONPATH_FIELD_OPT_KEY="hoodie.datasource.write.partitionpath.field"

#Incremental Constants
VIEW_TYPE_OPT_KEY="hoodie.datasource.view.type"
BEGIN_INSTANTTIME_OPT_KEY="hoodie.datasource.read.begin.instanttime"
VIEW_TYPE_INCREMENTAL_OPT_VAL="incremental"
END_INSTANTTIME_OPT_KEY="hoodie.datasource.read.end.instanttime"

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
10,application_1648194189527_0012,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Functions to create JSON data and Spark dataframe from this data

### Merge On Read 

For near real-time applications that mandate quick upserts, MERGE_ON_READ table type would be better suited. MOR table stores incoming upserts for each file group, onto a row based delta log (In Avro file format). This log is then merged with the existing Parquet file using a compactor during reads. 

Make sure you update the bucket name to your unique bucket before running this cell.

In [3]:
## CHANGE ME ##
config = {
    "table_name": "hudi_trips_table_mor",
    "target": "s3://<Your S3 Bucket Here>/hudi/hudi_trips_table_mor/",
    "primary_key": "trip_id",
    "sort_key": "tstamp",
    "commits_to_retain": "3",
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
STORAGE_TYPE_OPT_KEY="hoodie.datasource.write.storage.type"
COMPACTION_INLINE_OPT_KEY="hoodie.compact.inline"
COMPACTION_MAX_DELTA_COMMITS_OPT_KEY="hoodie.compact.inline.max.delta.commits"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
## Generates Data

from datetime import datetime

def get_json_data(start, count, dest):
    time_stamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    data = [{"trip_id": i, "tstamp": time_stamp, "route_id": chr(65 + (i % 10)), "destination": dest[i%10]} for i in range(start, start + count)]
    return data

# Creates the Dataframe
def create_json_df(spark, data):
    sc = spark.sparkContext
    return spark.read.json(sc.parallelize(data))


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


Let's generate 2M records to load into our Data Lake:

In [6]:
mor_dest = ["Seattle", "New York", "New Jersey", "Los Angeles", "Las Vegas", "Tucson","Washington DC","Philadelphia","Miami","San Francisco"]
df2 = create_json_df(spark, get_json_data(0, 2000000, mor_dest))
df2.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+--------+-------+-------------------+
|  destination|route_id|trip_id|             tstamp|
+-------------+--------+-------+-------------------+
|      Seattle|       A|      0|2022-03-27 07:04:39|
|     New York|       B|      1|2022-03-27 07:04:39|
|   New Jersey|       C|      2|2022-03-27 07:04:39|
|  Los Angeles|       D|      3|2022-03-27 07:04:39|
|    Las Vegas|       E|      4|2022-03-27 07:04:39|
|       Tucson|       F|      5|2022-03-27 07:04:39|
|Washington DC|       G|      6|2022-03-27 07:04:39|
| Philadelphia|       H|      7|2022-03-27 07:04:39|
|        Miami|       I|      8|2022-03-27 07:04:39|
|San Francisco|       J|      9|2022-03-27 07:04:39|
|      Seattle|       A|     10|2022-03-27 07:04:39|
|     New York|       B|     11|2022-03-27 07:04:39|
|   New Jersey|       C|     12|2022-03-27 07:04:39|
|  Los Angeles|       D|     13|2022-03-27 07:04:39|
|    Las Vegas|       E|     14|2022-03-27 07:04:39|
|       Tucson|       F|     15|2022-03-27 07:

## Bulk Insert the Initial Dataset

We will be using the Merge on Read storage option(STORAGE_TYPE_OPT_KEY, "MERGE_ON_READ") which  is to be explicitly set.

In [7]:
(df2.write.format(HUDI_FORMAT)
      .option(PRECOMBINE_FIELD_OPT_KEY, config["sort_key"])
      .option(RECORDKEY_FIELD_OPT_KEY, config["primary_key"])
      .option(TABLE_NAME, config['table_name'])
      .option(OPERATION_OPT_KEY, BULK_INSERT_OPERATION_OPT_VAL)
      .option(BULK_INSERT_PARALLELISM, 3)
      .option(HIVE_TABLE_OPT_KEY,config['table_name'])
      .option(HIVE_SYNC_ENABLED_OPT_KEY,"true")
      .option(HIVE_PARTITION_EXTRACTOR_CLASS_OPT_KEY,NONPARTITION_EXTRACTOR_CLASS_OPT_VAL)
      .option(KEYGENERATOR_CLASS_OPT_KEY,NONPARTITIONED_KEYGENERATOR_CLASS_OPT_VAL)
      .option(STORAGE_TYPE_OPT_KEY, "MERGE_ON_READ")
      .option(COMPACTION_INLINE_OPT_KEY, "false")
      .option(COMPACTION_MAX_DELTA_COMMITS_OPT_KEY, "3")
      .mode("Overwrite")
      .save(config['target']))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Verify the number of files 

Let us check the contents of S3 path. Bulk insert operation on Copy-On-Write and Merge-On-Read tables is identical in terms of performance. 

```
$ aws s3 ls s3://< S3 Bucket >/hudi/hudi_mor_trips_table/
                           PRE .hoodie/
2020-04-28 23:30:20          0 .hoodie_$folder$
2020-04-28 23:30:26         93 .hoodie_partition_metadata
2020-04-28 23:30:33    4378000 45b1ce07-f9ac-496d-8b03-20af011a0c44-0_1-194-3566_20200428233020.parquet
2020-04-28 23:30:34    5048941 932d5e97-c5f0-4c91-a7f6-f65d487a5e2b-0_2-194-3567_20200428233020.parquet
2020-04-28 23:30:34    5065824 ea6e8bfa-e70c-4f7e-90ec-37d018fb0acf-0_0-194-3565_20200428233020.parquet
```

Notice the delta commits 

```
$ aws s3 ls s3://< S3 Bucket >/hudi/hudi_mor_trips_table/.hoodie/
2020-04-28 23:30:21          0 .aux_$folder$
2020-04-28 23:30:21          0 .temp_$folder$
2020-04-28 23:30:37       1077 20200428233020.clean
2020-04-28 23:30:36       4929 20200428233020.deltacommit
2020-04-28 23:30:21          0 archived_$folder$
2020-04-28 23:30:21        264 hoodie.properties
```

This is the first commit 

## Batch Upsert some records


Now let us try to upsert some records into MOR table

In [8]:
upsert_dest = ["San Diego", "San Diego", "San Diego", "San Diego", "San Diego","San Diego","San Diego","San Diego","San Diego","San Diego"]
df3 = create_json_df(spark, get_json_data(1000000, 10, upsert_dest))
df3.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+--------+-------+-------------------+
|destination|route_id|trip_id|             tstamp|
+-----------+--------+-------+-------------------+
|  San Diego|       A|1000000|2022-03-27 07:05:58|
|  San Diego|       B|1000001|2022-03-27 07:05:58|
|  San Diego|       C|1000002|2022-03-27 07:05:58|
|  San Diego|       D|1000003|2022-03-27 07:05:58|
|  San Diego|       E|1000004|2022-03-27 07:05:58|
|  San Diego|       F|1000005|2022-03-27 07:05:58|
|  San Diego|       G|1000006|2022-03-27 07:05:58|
|  San Diego|       H|1000007|2022-03-27 07:05:58|
|  San Diego|       I|1000008|2022-03-27 07:05:58|
|  San Diego|       J|1000009|2022-03-27 07:05:58|
+-----------+--------+-------+-------------------+

In [9]:
(df3.write.format(HUDI_FORMAT)
      .option(PRECOMBINE_FIELD_OPT_KEY, config["sort_key"])
      .option(RECORDKEY_FIELD_OPT_KEY, config["primary_key"])
      .option(TABLE_NAME, config['table_name'])
      .option(OPERATION_OPT_KEY, UPSERT_OPERATION_OPT_VAL)
      .option(UPSERT_PARALLELISM, 20)
      .option(HUDI_CLEANER_POLICY, KEEP_LATEST_COMMITS)
      .option(HUDI_COMMITS_RETAINED,config["commits_to_retain"])
      .option(HIVE_TABLE_OPT_KEY,config['table_name'])
      .option(HIVE_SYNC_ENABLED_OPT_KEY,"true")
      .option(HIVE_PARTITION_EXTRACTOR_CLASS_OPT_KEY,NONPARTITION_EXTRACTOR_CLASS_OPT_VAL)
      .option(KEYGENERATOR_CLASS_OPT_KEY,NONPARTITIONED_KEYGENERATOR_CLASS_OPT_VAL)  
      .option(STORAGE_TYPE_OPT_KEY, "MERGE_ON_READ")
      .option(COMPACTION_INLINE_OPT_KEY, "false")
      .option(COMPACTION_MAX_DELTA_COMMITS_OPT_KEY, "3")
      .mode("Append")
      .save(config['target']))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Verify the number of tables created


In [10]:
spark.sql("show tables").show(20,False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+------------------------------+-----------+
|database|tableName                     |isTemporary|
+--------+------------------------------+-----------+
|default |hudi_partitioned_trips_table  |false      |
|default |hudi_trips_table_mor_ro       |false      |
|default |hudi_trips_table_mor_rt       |false      |
|default |hudi_trips_table_v2           |false      |
|default |sales_order_detail_hudi_cow   |false      |
|default |sales_order_detail_hudi_mor_ro|false      |
|default |sales_order_detail_hudi_mor_rt|false      |
+--------+------------------------------+-----------+


We can observe that there are two tables _ro and _rt created for  Merge On Read storage option. 

1.hudi_mor_trips_table_ro     
2.hudi_mor_trips_table_rt  


**Read optimized view** provides the latest compacted dataset from MoR tables.
Lets query the table  -> hudi_mor_trips_table_ro . Since the table is not yet compacted, upserted records will not be reflected in _ro table

In [11]:
spark.sql("select trip_id, route_id, destination, tstamp from "+config['table_name']+"_ro where trip_id between 999996 and 1000010").show(20,False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------+-------------+-------------------+
|trip_id|route_id|destination  |tstamp             |
+-------+--------+-------------+-------------------+
|1000000|A       |Seattle      |2022-03-27 07:04:39|
|1000001|B       |New York     |2022-03-27 07:04:39|
|1000002|C       |New Jersey   |2022-03-27 07:04:39|
|1000003|D       |Los Angeles  |2022-03-27 07:04:39|
|1000004|E       |Las Vegas    |2022-03-27 07:04:39|
|1000005|F       |Tucson       |2022-03-27 07:04:39|
|1000006|G       |Washington DC|2022-03-27 07:04:39|
|1000007|H       |Philadelphia |2022-03-27 07:04:39|
|1000008|I       |Miami        |2022-03-27 07:04:39|
|1000009|J       |San Francisco|2022-03-27 07:04:39|
|1000010|A       |Seattle      |2022-03-27 07:04:39|
|999996 |G       |Washington DC|2022-03-27 07:04:39|
|999997 |H       |Philadelphia |2022-03-27 07:04:39|
|999998 |I       |Miami        |2022-03-27 07:04:39|
|999999 |J       |San Francisco|2022-03-27 07:04:39|
+-------+--------+-------------+--------------

**Real time view** provides the latest committed data from a MoR table by merging the columnar and row-based files inline.
Lets query the real-time table -> hudi_mor_trips_table_rt

In [12]:
spark.sql("select trip_id, route_id, destination, tstamp from "+config['table_name']+"_rt where trip_id between 999996 and 1000010").show(20,False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------+-------------+-------------------+
|trip_id|route_id|destination  |tstamp             |
+-------+--------+-------------+-------------------+
|1000000|A       |San Diego    |2022-03-27 07:05:58|
|1000001|B       |San Diego    |2022-03-27 07:05:58|
|1000002|C       |San Diego    |2022-03-27 07:05:58|
|1000003|D       |San Diego    |2022-03-27 07:05:58|
|1000004|E       |San Diego    |2022-03-27 07:05:58|
|1000005|F       |San Diego    |2022-03-27 07:05:58|
|1000006|G       |San Diego    |2022-03-27 07:05:58|
|1000007|H       |San Diego    |2022-03-27 07:05:58|
|1000008|I       |San Diego    |2022-03-27 07:05:58|
|1000009|J       |San Diego    |2022-03-27 07:05:58|
|1000010|A       |Seattle      |2022-03-27 07:04:39|
|999996 |G       |Washington DC|2022-03-27 07:04:39|
|999997 |H       |Philadelphia |2022-03-27 07:04:39|
|999998 |I       |Miami        |2022-03-27 07:04:39|
|999999 |J       |San Francisco|2022-03-27 07:04:39|
+-------+--------+-------------+--------------

Check the S3 path again. There is no change in number of Parquet files after upsert operation unlike Copy-On-Write tables. However, you will notice a new delta file created for the upsert operation

```
$ aws s3 ls s3://<Your S3 Bucket Here>/hudi/hudi_trips_table_mor/
                           PRE .hoodie/
2020-04-28 23:33:22       2071 .ea6e8bfa-e70c-4f7e-90ec-37d018fb0acf-0_20200428233020.log.1_0-227-3837
2020-04-28 23:30:20          0 .hoodie_$folder$
2020-04-28 23:30:26         93 .hoodie_partition_metadata
2020-04-28 23:30:33    4378000 45b1ce07-f9ac-496d-8b03-20af011a0c44-0_1-194-3566_20200428233020.parquet
2020-04-28 23:30:34    5048941 932d5e97-c5f0-4c91-a7f6-f65d487a5e2b-0_2-194-3567_20200428233020.parquet
2020-04-28 23:30:34    5065824 ea6e8bfa-e70c-4f7e-90ec-37d018fb0acf-0_0-194-3565_20200428233020.parquet

```


## Deleting Records.

Apache Hudi supports implementing two types of deletes on data stored in Hudi datasets, by enabling the user to specify a different record payload implementation.

* **Soft Deletes** : With soft deletes, user wants to retain the key but just null out the values for all other fields. This can be simply achieved by ensuring the appropriate fields are nullable in the dataset schema and simply upserting the dataset after setting these fields to null.
    
* **Hard Deletes** : A stronger form of delete is to physically remove any trace of the record from the dataset. 

Let's now execute some hard delete operations on our dataset which will remove the records from our dataset.

Let's delete the 10 records with the "San Diego" destination we upserted to the table. Note that the only change is the single line that set the OPERATION_OPT_KEY to DELETE_OPERATION_OPT_VAL to remove all records in the dataset you submit

```
.option(OPERATION_OPT_KEY, DELETE_OPERATION_OPT_VAL)
```

In [13]:
(df3.write.format(HUDI_FORMAT)
      .option(PRECOMBINE_FIELD_OPT_KEY, config["sort_key"])
      .option(RECORDKEY_FIELD_OPT_KEY, config["primary_key"])
      .option(TABLE_NAME, config['table_name'])
      .option(OPERATION_OPT_KEY, DELETE_OPERATION_OPT_VAL)
      .option(UPSERT_PARALLELISM, 20)
      .option(HUDI_CLEANER_POLICY, KEEP_LATEST_COMMITS)
      .option(HUDI_COMMITS_RETAINED,config["commits_to_retain"])
      .option(HIVE_TABLE_OPT_KEY,config['table_name'])
      .option(HIVE_SYNC_ENABLED_OPT_KEY,"true")
      .option(HIVE_PARTITION_EXTRACTOR_CLASS_OPT_KEY,NONPARTITION_EXTRACTOR_CLASS_OPT_VAL)
      .option(KEYGENERATOR_CLASS_OPT_KEY,NONPARTITIONED_KEYGENERATOR_CLASS_OPT_VAL)
      .option(STORAGE_TYPE_OPT_KEY, "MERGE_ON_READ")
      .mode("Append")
      .save(config['target']))



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


Query and observe that the updated records are reflected in only _rt table as it gives the most lastest snapshot of data. While, _ro table still contains the deleted rows.

1.hudi_mor_trips_table_ro    
2.hudi_mor_trips_table_rt  


In [14]:
spark.sql("select trip_id, route_id, destination, tstamp from "+config['table_name'] +"_ro where trip_id between 999996 and 1000010").show(20,False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------+-------------+-------------------+
|trip_id|route_id|destination  |tstamp             |
+-------+--------+-------------+-------------------+
|1000000|A       |Seattle      |2022-03-27 07:04:39|
|1000001|B       |New York     |2022-03-27 07:04:39|
|1000002|C       |New Jersey   |2022-03-27 07:04:39|
|1000003|D       |Los Angeles  |2022-03-27 07:04:39|
|1000004|E       |Las Vegas    |2022-03-27 07:04:39|
|1000005|F       |Tucson       |2022-03-27 07:04:39|
|1000006|G       |Washington DC|2022-03-27 07:04:39|
|1000007|H       |Philadelphia |2022-03-27 07:04:39|
|1000008|I       |Miami        |2022-03-27 07:04:39|
|1000009|J       |San Francisco|2022-03-27 07:04:39|
|1000010|A       |Seattle      |2022-03-27 07:04:39|
|999996 |G       |Washington DC|2022-03-27 07:04:39|
|999997 |H       |Philadelphia |2022-03-27 07:04:39|
|999998 |I       |Miami        |2022-03-27 07:04:39|
|999999 |J       |San Francisco|2022-03-27 07:04:39|
+-------+--------+-------------+--------------

In [15]:
spark.sql("select trip_id, route_id, destination, tstamp from "+config['table_name']+"_rt where trip_id between 999996 and 1000010").show(20,False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------+-------------+-------------------+
|trip_id|route_id|destination  |tstamp             |
+-------+--------+-------------+-------------------+
|1000010|A       |Seattle      |2022-03-27 07:04:39|
|999996 |G       |Washington DC|2022-03-27 07:04:39|
|999997 |H       |Philadelphia |2022-03-27 07:04:39|
|999998 |I       |Miami        |2022-03-27 07:04:39|
|999999 |J       |San Francisco|2022-03-27 07:04:39|
+-------+--------+-------------+-------------------+


Check the count of records in the two tables after deletion. 

1.hudi_mor_trips_table_ro     
2.hudi_mor_trips_table_rt  


Observe that total record count indicates that 10 upserted records have been deleted only from _rt table

In [16]:
spark.sql("select count(*) from "+config['table_name']+"_ro" ).show(20,False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+
|count(1)|
+--------+
|2000000 |
+--------+

In [17]:
spark.sql("select count(*) from "+config['table_name']+"_rt" ).show(20,False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+
|count(1)|
+--------+
|1999990 |
+--------+

## Hudi Compaction

Compaction refers to background activity to reconcile differential data structures within Hudi e.g: moving updates from row based log files to columnar formats. Internally, compaction manifests as a special commit on the timeline. With Merge_On_Read Table, Hudi ingestion needs to also take care of compacting delta files. Compaction can be performed in an asynchronous-mode by letting compaction run concurrently with ingestion or in a serial fashion with one after another.

- **Inline compaction:** For Merge On Read table types, inline compaction is turned on by default which runs after every ingestion. The compaction frequency can be changed by setting the property **hoodie.compact.inline.max.delta.commits**

- **Manual Compaction:** Inline compaction works well for most of the use cases. However, there can be few latency sensitive cases, where you would want to disable inline compaction and run it manually.



In [18]:
%%html
<style>
table {float:left}
</style>

We will now see how to schedule and run manual compaction on hudi table and how that impacts read optimized tables.Before we start, we need to have avro schema file. Copy and paste below schema on a notepad and save the file as *morHudiSchema.avsc* . Now place *morHudiSchema.avsc* to an appropriate S3 bucket, on which you have read and write access.

Lets open Hudi cli and schedule a manual compaction 

/usr/lib/hudi/cli/bin/hudi-cli.sh

hudi->connect --path s3://< S3 Bucket >/hudi/hudi_trips_table_mor/

hudi:hudi_trips_table_mor->compaction schedule --hoodieConfigs hoodie.compact.inline.max.delta.commits=1

hudi:hudi_trips_table_mor->compactions show all

|  Compaction Instant Time | State     | Total FileIds to be Compacted |
| -------------------------| ----------|-------------------------------|
| 20211110000358           | REQUESTED | 1                             |

Now that we have scheduled a compaction. Lets go ahead and run the compaction job. 

Refresh the metadata by running connect command again

hudi->connect --path s3://< S3 Bucket >/hudi/hudi_trips_table_mor/

hudi:hudi_trips_table_mor->compaction run --parallelism 2 --schemaFilePath s3://your-s3-bucket/morHudiSchema.avsc

hudi:hudi_trips_table_mor->compactions show all

|  Compaction Instant Time | State     | Total FileIds to be Compacted |
| -------------------------| ----------|-------------------------------|
| 20211110000358           | COMPLETED | 1                             |


 

Note:- If you do not see state as COMPLETED, run the connect --path again to refresh the metadata.


Notice the difference in count after we have run the compaction job. Both _ro and _rt tables have now latest snapshot of data.

In [19]:
spark.sql("select count(*) from "+config['table_name']+"_ro" ).show(20,False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+
|count(1)|
+--------+
|1999990 |
+--------+

In [20]:
spark.sql("select count(*) from "+config['table_name']+"_rt" ).show(20,False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+
|count(1)|
+--------+
|1999990 |
+--------+

In [26]:
spark.sql("select trip_id, route_id, destination, tstamp from "+config['table_name']+"_ro where trip_id between 999996 and 1000010").show(20,False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------+-------------+-------------------+
|trip_id|route_id|destination  |tstamp             |
+-------+--------+-------------+-------------------+
|1000010|A       |Seattle      |2022-03-27 07:04:39|
|999996 |G       |Washington DC|2022-03-27 07:04:39|
|999997 |H       |Philadelphia |2022-03-27 07:04:39|
|999998 |I       |Miami        |2022-03-27 07:04:39|
|999999 |J       |San Francisco|2022-03-27 07:04:39|
+-------+--------+-------------+-------------------+

In [15]:
spark.sql("select trip_id, route_id, destination, tstamp from "+config['table_name']+"_rt where trip_id between 999996 and 1000010").show(20,False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------+-------------+-------------------+
|trip_id|route_id|destination  |tstamp             |
+-------+--------+-------------+-------------------+
|1000010|A       |Seattle      |2022-03-27 07:04:39|
|999996 |G       |Washington DC|2022-03-27 07:04:39|
|999997 |H       |Philadelphia |2022-03-27 07:04:39|
|999998 |I       |Miami        |2022-03-27 07:04:39|
|999999 |J       |San Francisco|2022-03-27 07:04:39|
+-------+--------+-------------+-------------------+

Optionally, you can run the same queries on Athena.